# Generate Actor Dataframes
This code will load the actors list from the wikipedia scraper JSON file, create a dataframe with them. Then we create a second dataframe from the CSV file of black actors, also from wikipedia, and we use regex to remove some of the elements that this one has that are irrelevant and must be removed to compare both dataframes. Lastly we do the comparison, giving a table with the actors name, and a 0 if they are no black, a 1 if they are

In [12]:
import pandas as pd
import json
from pandas import json_normalize
import os
import regex as re

#os.path.normpath(path/to/dir)
# Or relative path, does not work sometimes, just uncomment it and comment the complete pathone
# actor_df = pd.read_json('data\metadata\movies_metadata.json', encoding='latin-1')
df = pd.read_json('D:\ITU\DataintheWild\Project\Deprecated_DITW_project-master\data\metadata\movies_metadata.json', encoding='latin-1')
actor_df = (df[df.index.isin( ['actors','title'])]
            .T
            .explode('actors')
            .reset_index())

actor_df

,index,title,actors
0,1,Exodus: Gods and Kings,Christian Bale
1,1,Exodus: Gods and Kings,Joel Edgerton
2,1,Exodus: Gods and Kings,John Turturro
3,1,Exodus: Gods and Kings,Aaron Paul
4,1,Exodus: Gods and Kings,Ben Mendelsohn
5,1,Exodus: Gods and Kings,Sigourney Weaver
6,1,Exodus: Gods and Kings,Ben Kingsley
7,2,The Little Mermaid (2023 film),Halle Bailey
8,2,The Little Mermaid (2023 film),Jonah Hauer-King
9,2,The Little Mermaid (2023 film),Daveed Diggs


In [13]:
# Create second dataframe, with colored actors
# relative path, comment the other one if wish to change
#black_actor_path = 'data/actors.csv' 
black_actor_path = 'D:/ITU/DataintheWild/Project/Deprecated_DITW_project-master/data/actors.csv'

black_actor_df = pd.read_csv(black_actor_path, on_bad_lines='skip')
black_actor_df.head()

,id
0,Ernest Hogan – 1865
1,Matilda Sissieretta Joyner Jones (also known a...
2,George H. Reed – 1866
3,Bob Cole – 1868
4,Madame Sul-Te-Wan – 1873


In [14]:
actor_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44 entries, 0 to 43
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   index   44 non-null     int64 
 1   title   44 non-null     object
 2   actors  44 non-null     object
dtypes: int64(1), object(2)
memory usage: 1.2+ KB


In [15]:
regex = '|'.join(map(re.escape, actor_df['actors']))

actors = black_actor_df['id'].str.extract(f'({regex})', expand=False).dropna()

labeled_actor_df = actor_df.assign(indf=actor_df['actors'].isin(actors).astype(int))

In [16]:
labeled_actor_df

,index,title,actors,indf
0,1,Exodus: Gods and Kings,Christian Bale,0
1,1,Exodus: Gods and Kings,Joel Edgerton,0
2,1,Exodus: Gods and Kings,John Turturro,0
3,1,Exodus: Gods and Kings,Aaron Paul,0
4,1,Exodus: Gods and Kings,Ben Mendelsohn,0
5,1,Exodus: Gods and Kings,Sigourney Weaver,0
6,1,Exodus: Gods and Kings,Ben Kingsley,0
7,2,The Little Mermaid (2023 film),Halle Bailey,1
8,2,The Little Mermaid (2023 film),Jonah Hauer-King,0
9,2,The Little Mermaid (2023 film),Daveed Diggs,1


In [ ]:
#df.to_csv('data\actor_race.csv')  